Direct implementation of actor critic method as done in https://www.youtube.com/watch?v=G0L8SN02clA&t

Actor critic methods are a way of reducing the amount of variance from a policy gradient. The idea is that by using two networks, or at least two different function approximators, one can learn two more easily digestible curriculums which can help an agent successfully master whatever environment it is currently in. 

To motivate actor critic algorithms mathematically, recall the policy gradient is  

$$\nabla_{\theta} J(\theta) = \frac{1}{N} \sum_{i=1}^N \sum_{t=1}^T \nabla_{\theta} log \pi_{\theta}(a_{i,t} | s_{i,t}) G_{i,t}$$

Where $G_{i,t}$ represents some advantage/reward-to-go this policy takes has compared to the other possible actions at that state. In the regular old policy-gradient, $G$ is simply the total remaining rewards to go, implying that this action could either be really good or really bad depending on the value of $G$. 

Unfortunately, there are many caveats associated with $G$ simply being the total reward to go. For one, this value is  insensitive to a relative reward difference, meaning that if you were to simply add $40$ to all possible rewards the agent receives, you could end up with a very different policy! Good policies should not be sensitive to scalar shifts. Clearly, we have some work to do... 

There are a bunch of ways to fix $G_{i,t}$ for variance reduction, where one of the most popular (and already implemented) was a baseline which corresponded to the mean reward. Additionally, we could see how something like an expected reward to go, would be a better estimate and variance reduction technique. We'll see that shortly.

With a baseline, the policy gradient is 

$$\nabla_{\theta} J(\theta) = \frac{1}{N} \sum_{i=1}^N \sum_{t=1}^T \nabla_{\theta} log \pi_{\theta}(a_{i,t} | s_{i,t})( G_{i,t} - b(s_{i,t})$$

However, there is no reason for this baseline to be a constant number! In fact, it would be a great idea if this baseline could also be a function of the state observation $s_t$. At the same time, this value is starting to look a lot like the reinforcement learning concepts we considered at the beginning of this course, mainly, the $Q$ function, the $V$ function, and the $A$ function. 

The $Q$ value is the expected reward to go, given a state and action. 

$Q(s_t,a_t) = E[G_t | (s_t, a_t)]$

The $V$ value is the average $Q$ values in a given state, over possible actions. 

$V(s_t) = \sum_a Q(s,a)$

and $A$ represents the advantage a particular $Q$ value has over the average rewards to go in a state. It is simply 

$A = Q - V$


Any three of these functions, (Q,V,A) could be represented as some combination of the other one, and thus could be used as the approximator we plug into the AC algorithm.

In any case we can leverage Bellman optimality conditions, and use that as a method to fit this critic network. 

The code for this is below. More specifically, how to make an Actor-Critic Algorithm that trains an actor-critic agent in a continuous action space.

Spoiler alert: This algorithm won't work! But that just gives more motivation to making my own implementation. 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym 
#from gym import wrappers
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
# Two networks, with two hidden layers. 

#we'll have an actor network, and a critic network.

class GenericNetwork(nn.Module):
    # To use for both networks 
    def __init__(self,lr,input_dims,fc1_dims,fc2_dims,
                n_actions):
        super(GenericNetwork,self).__init__()
        
        self.lr = lr
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
    
        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims,self.fc2_dims)
        
        self.fc3 = nn.Linear(self.fc2_dims,self.n_actions)
        
        #basis network for both A and C, # of output will change!
        
        self.optimizer = torch.optim.Adam(self.parameters(),lr = self.lr)
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
        self.to(self.device)
    #now the forward pass!
    
    def forward(self,state): 
        #state and obs are interchangeable, not a POMDP setting...
        state = torch.Tensor(state).to(self.device)        
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) #no activation, handled later 
        return x

In [31]:
class Agent(object):
    def __init__(self,alpha,beta,input_dims,gamma=0.99, n_actions = 2, 
                layer1_size=64,layer2_size = 64,n_outputs=1 ):
        self.gamma = gamma
        self.log_probs = None # log probability of selecting an action 
        self.n_outputs = n_outputs
        self.actor = GenericNetwork(alpha,input_dims,layer1_size,layer2_size,n_actions = n_actions)
        self.critic = GenericNetwork(beta,input_dims,layer1_size,layer2_size,n_actions = 1) 
          
    def choose_action(self,observation):
        mu, sigma = self.actor.forward(observation)
        # now we need to force these values to make sense, i.e make std real and non-zero. 
        sigma = torch.exp(sigma)
        action_probs = torch.distributions.Normal(mu,sigma) #calculate dist
        #draw sample from it 
        probs = action_probs.sample(sample_shape = torch.Size([self.n_outputs])) #gives us single sample output 
        self.log_probs = action_probs.log_prob(probs).to(self.actor.device)
        action = torch.tanh(probs) #again bound between -1 and 1 to fit environment 
        return action.item() #not a tensor!
    
    def learn(self,state,reward,new_state,done):
        #temporal difference style, using state at current and next state.
        #PG doesn't use that, Monte-carlo style, which accumulates rewards over the episode. 
        #This is just a state by state 
        self.actor.optimizer.zero_grad()
        self.critic.optimizer.zero_grad()
        
        critic_value_ = self.critic.forward(new_state) #Q value of next state for TD critic loss
        critic_value = self.critic.forward(state) #Q val at current state
        
        reward = torch.Tensor([reward]).to(self.actor.device)
        
        delta = reward + self.gamma* critic_value_*(1-int(done))  - critic_value#loss!
            #y_i - Vphi i , and this is the same as advantage
        
        actor_loss = -self.log_probs * delta
        
        critic_loss = delta **2  #minimize 
        
        #now sum the two, and backpropagate
        
        (actor_loss + critic_loss).backward() #can't do two backward passes at once, but these losses
        #are inpdt so it will update separately :) 
        self.actor.optimizer.step()
        self.critic.optimizer.step()
        
        

In [34]:
# now the main

agent = Agent(alpha = 0.00005, beta = .00001, input_dims = [2], gamma=0.99,
             layer1_size = 256, layer2_size=256)

env = gym.make('MountainCarContinuous-v0')
score_history = []
num_episodes = 100
for i in range(num_episodes):
    done = False
    score = 0
    observation = env.reset()
    #now the game!
    
    while not done:
        action = np.array(agent.choose_action(observation)).reshape(1,)
        env.render()
        observation_,reward, done, _ = env.step(action)
        agent.learn(observation,reward,observation_, done)
        
        observation = observation_        
        score += reward
    score_history.append(score)
    
    print("Episode ", i, 'score ', score)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode  0 score  -16.779608691465885
Episode  1 score  -2.6548709969667974
Episode  2 score  -1.0745859013155923
Episode  3 score  -0.5962055210359678
Episode  4 score  -1.149691101649804
Episode  5 score  -7.181106739325993
Episode  6 score  -3.1848526988643067
Episode  7 score  -4.306317446703823
Episode  8 score  -11.762729564261308
Episode  9 score  -9.006712802846263
Episode  10 score  -0.2967319467658178
Episode  11 score  -0.3471004144209088
Episode  12 score  -0.553946355824586
Episode  13 score  -2.430584147925059
Episode  14 score  -0.47707715021474617
Episode  15 score  -1.3979827336555242
Episode  16 score  -3.1887258379778367
Episode  17 score  -9.217684679076903
Episode  18 score  -1.5369512304840276
Episode  19 score  -9.681950230101256
Episode  20 score  -3.0610717729908337


So why doesn't this win? It only ever learns to minimize the negative reward. 


It learns quite hilariously, that if it stays at the bottom, the negative reward is minimized!

This is just one example of why AC and RL in general is a shaky tool!

